# Multi i-TED: CRT study
## Fourth: More combinations and plane coincidence

- Study of CRT using different algorithms:
    - First event (SKEW=0)
    - Average of the first n events (DEGREE=0)
    - Energy-weighted average of the first n events (DEGREE=1)
    - Second power of the enery-weighted average of the first n events (DEGREE=2)
    - Third power of the enery-weighted average of the first n events (DEGREE=3)
- The measurement using Na22 for timing studies:
    - It includes filters around the 511keV energy peak applied per crystal
    - Coincidences between the scatterer and all of the absorbers are taken into account
    - Gaussian fits are used to determine the standard deviation
- The calculation of the timing:
    - The timing weighted is calculated as energy^DEGREE
    - In case energy^DEGREE causes an error (`floating point exception`), then it is considered 0 and not taking into account for the average
    - In case the sum of the energy^DEGREE is 0, then the time will be 0, as all weights were too low

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import copy

uproot              4.3.5
Welcome to JupyROOT 6.28/02
sklearn             1.2.2
torch               2.0.0
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
file = lambda skew, degree: f"/run/media/bgameiro/92A6-F521/Multi_iTED_Timing/Na22_iTEDD_timing_D.2023_04_18_T.11_31_52_C.itedABCD_lab_2023.02.22_4.0v_888_120s_CW100_{skew}_{degree}.root"

In [4]:
skew = [i+1 for i in range(16)]
degree = [0,1,2,3]

spectra = pandas.DataFrame(index = skew, columns = degree)

In [5]:
for i_skew in skew:
    for i_degree in degree:
        f_root = ROOT.TFile.Open(file(i_skew,i_degree))
        t_root = f_root.COINCIDENCES
        
        canvas = ROOT.TCanvas()
        canvas.cd()
        
        t_root.Draw("Delta_t[15]-Delta_t[16]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        h_root = ROOT.gPad.GetPrimitive("htemp")
        
        #t_root.Draw("Delta_t[15]-Delta_t[17]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root1 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root1)
        
        #t_root.Draw("Delta_t[15]-Delta_t[18]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root2 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root2)
        
        #t_root.Draw("Delta_t[15]-Delta_t[19]","deposited_energy[15]>160.0&deposited_energy[15]<195.0&&deposited_energy[16]>125.0&deposited_energy[16]<165.0","")
        #h_root3 = ROOT.gPad.GetPrimitive("htemp")
        #h_root.Add(h_root2)
        
        h_root.Draw()
                
        latex = ROOT.TLatex()
        latex.SetNDC()
        latex.SetTextSize(0.03)
        latex.DrawText(0.25, 0.75, f"Skew:{i_skew}")
        latex.DrawText(0.25, 0.7, f"Degree:{i_degree}")        
        
        gaussFit = ROOT.TF1("gaussFit", "gaus", -5.0, 5.0)
        h_root.Fit(gaussFit,"QR")
    
        sigma = abs(gaussFit.GetParameter(2))
        centroid_ch = gaussFit.GetParameter(1)
        
        spectra[i_degree][i_skew] = {
            "histo":canvas,
            "mean":h_root.GetMean(),
            "mean_fit":centroid_ch,
            "std":h_root.GetStdDev(),
            "std_fit":sigma,
            "skew":h_root.GetSkewness(),
            "kurt":h_root.GetKurtosis(),
        }

## Analysis

### Standard deviation

In [6]:
spectra.applymap(lambda x: x["std_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,1.461833,1.461833,1.461833,1.461833
2,1.353935,1.446548,1.523530,1.496958
3,1.285825,1.433603,1.481470,1.487713
4,1.231788,1.417217,1.472548,1.492461
5,1.166858,1.398856,1.462067,1.504304
6,1.110786,1.376599,1.459354,1.543133
7,1.067434,1.342077,1.464800,1.581459
8,1.033222,1.318095,1.475755,1.626040
9,1.017052,1.285803,1.490974,1.667020
10,1.017445,1.251463,1.500214,1.721150


In [7]:
spectra.applymap(lambda x: x["std"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,1.723254,1.723254,1.723254,1.723254
2,1.850902,2.220804,1.960248,1.779143
3,2.003701,2.215663,1.874325,1.740803
4,2.105275,2.127280,1.794819,1.713594
5,2.172813,2.009810,1.745051,1.686730
6,2.253340,1.922061,1.708771,1.703135
7,2.362952,1.840037,1.695277,1.724941
8,2.388269,1.751664,1.685457,1.760897
9,2.321521,1.681038,1.685813,1.795132
10,2.208307,1.610165,1.685929,1.820711


### Mean value, skewness, and kurtosis

In [8]:
spectra.applymap(lambda x: x["mean_fit"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,-0.650543,-0.650543,-0.650543,-0.650543
2,-0.719473,-0.765986,-0.797317,-0.777255
3,-0.755937,-0.828858,-0.845725,-0.842560
4,-0.783197,-0.872681,-0.894242,-0.903176
5,-0.783704,-0.896462,-0.934571,-0.958438
6,-0.770959,-0.908688,-0.966464,-1.011811
7,-0.753980,-0.913734,-0.997270,-1.058682
8,-0.729392,-0.915269,-1.023513,-1.103910
9,-0.695171,-0.911475,-1.046051,-1.149260
10,-0.655495,-0.898742,-1.068238,-1.194389


In [9]:
spectra.applymap(lambda x: x["skew"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,0.108488,0.108488,0.108488,0.108488
2,0.117829,0.078990,0.302431,0.372200
3,0.060835,0.114138,0.373605,0.276017
4,0.201416,0.264044,0.386125,0.426785
5,0.139670,0.343924,0.424563,0.385765
6,0.200081,0.326988,0.476954,0.439212
7,0.145276,0.311058,0.412891,0.426294
8,0.147138,0.317352,0.363078,0.429305
9,0.097788,0.333541,0.340775,0.386171
10,0.066025,0.319722,0.346376,0.385368


In [10]:
spectra.applymap(lambda x: x["kurt"]).style.background_gradient(cmap ='YlOrRd',axis=None)

,0,1,2,3
1,4.552298,4.552298,4.552298,4.552298
2,6.809468,5.152239,4.661985,4.367410
3,7.059212,4.676600,4.715399,4.171538
4,6.852992,4.756846,4.348892,3.706981
5,6.654678,4.956530,4.140099,2.805881
6,6.534908,5.065603,3.769525,2.640814
7,6.076792,5.222734,3.622329,2.425688
8,5.706663,5.181162,3.265025,2.223504
9,5.427044,5.195241,3.110456,2.040270
10,5.218802,5.061761,2.886934,1.701601


## Plots

Some notes:
- Although not noticeable it's worth taking into account the limitation of not being able to calculate very small numbers with c++'s pow
- Values of skewness are fairly low (<0.5) throughout
- Kurtosis decreases with the number of points and the degree of the weight

Improvements:
- Sum the histograms for all the absorbers to have a more complete picture of the timing between planes
- More combinations (more peaks, higher level weights)?

In [11]:
for i_skew in skew:
    for i_degree in degree:        
        spectra[i_degree][i_skew]["histo"].Draw()